<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-structure" data-toc-modified-id="Basic-structure-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic structure</a></span></li><li><span><a href="#Adding-an-updated-method" data-toc-modified-id="Adding-an-updated-method-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Adding an updated method</a></span></li><li><span><a href="#Set-threshold-and-number-of-simulations" data-toc-modified-id="Set-threshold-and-number-of-simulations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set threshold and number of simulations</a></span></li><li><span><a href="#Extract-the-results" data-toc-modified-id="Extract-the-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract the results</a></span></li></ul></div>

# Basic structure

In [1]:
from elfi.methods.parameter_inference import ParameterInference

class CustomMethod(ParameterInference):

    def __init__(self, model, output_names, **kwargs):
        super(CustomMethod, self).__init__(model, output_names, **kwargs)

    def set_objective(self):
        # Request 3 batches to be generated
        self.objective['n_batches'] = 3

    def extract_result(self):
        return self.state

In [2]:
import elfi.examples.ma2 as ma2

# Get a ready made MA2 model to test our inference method with
m = ma2.get_model()

# We want the outputs from node 'd' of the model `m` to be available
custom_method = CustomMethod(m, ['d'])

# Run the inference
custom_method.infer()  # {'n_batches': 3, 'n_sim': 3000}

{'n_batches': 3, 'n_sim': 3000}

# Adding an updated method

In [3]:
class CustomMethod(ParameterInference):
    def __init__(self, model, output_names, **kwargs):
        super(CustomMethod, self).__init__(model, output_names, **kwargs)

        # Hard code a threshold and discrepancy node name for now
        self.threshold = .1
        self.discrepancy_name = output_names[0]

        # Prepare lists to push the filtered outputs into
        self.state['filtered_outputs'] = {name: [] for name in output_names}

    def update(self, batch, batch_index):
        super(CustomMethod, self).update(batch, batch_index)

        # Make a filter mask (logical numpy array) from the distance array
        filter_mask = batch[self.discrepancy_name] <= self.threshold

        # Append the filtered parameters to their lists
        for name in self.output_names:
            values = batch[name]
            self.state['filtered_outputs'][name].append(values[filter_mask])

    # other methods as before
    def set_objective(self):
        # Request 3 batches to be generated
        self.objective['n_batches'] = 3

    def extract_result(self):
        return self.state

In [4]:
# Get a ready made MA2 model to test our inference method with
m = ma2.get_model()

# We want the outputs from node 'd' of the model `m` to be available
custom_method = CustomMethod(m, ['d'])

# Run the inference
custom_method.infer()

{'filtered_outputs': {'d': [array([ 0.04118507,  0.00667366,  0.09887394,  0.09115247,  0.01787308,
           0.09096578,  0.03155234]),
   array([ 0.09199846]),
   array([ 0.01787483,  0.05463452,  0.06729574,  0.043176  ,  0.01443466,
           0.09046322])]},
 'n_batches': 3,
 'n_sim': 3000}

# Set threshold and number of simulations

In [5]:
class CustomMethod(ParameterInference):
    def __init__(self, model, discrepancy_name, threshold, **kwargs):
        # Create a name list of nodes whose outputs we wish to receive
        output_names = [discrepancy_name] + model.parameter_names
        super(CustomMethod, self).__init__(model, output_names, **kwargs)

        self.threshold = threshold
        self.discrepancy_name = discrepancy_name

        # Prepare lists to push the filtered outputs into
        self.state['filtered_outputs'] = {name: [] for name in output_names}


    def update(self, batch, batch_index):
        super(CustomMethod, self).update(batch, batch_index)

        # Make a filter mask (logical numpy array) from the distance array
        filter_mask = batch[self.discrepancy_name] <= self.threshold

        # Append the filtered parameters to their lists
        for name in self.output_names:
            values = batch[name]
            self.state['filtered_outputs'][name].append(values[filter_mask])

    def set_objective(self, n_sim):
        self.objective['n_sim'] = n_sim

    def extract_result(self):
        return self.state

In [6]:
# Run the inference
custom_method = CustomMethod(m, 'd', threshold=.1, batch_size=1000)
custom_method.infer(n_sim=2000)  # {'n_batches': 2, 'n_sim': 2000, 'filtered_outputs': ...}

{'filtered_outputs': {'d': [array([ 0.07274003,  0.05496166,  0.03369005,  0.02783118,  0.06705382]),
   array([ 0.04938172,  0.039332  ,  0.04218729])],
  't1': [array([ 0.67284382,  0.76477027,  0.62571009,  0.46825312,  0.54522481]),
   array([ 0.60167879,  1.08463265,  0.8094352 ])],
  't2': [array([ 0.54927791,  0.45852012,  0.27436906,  0.75507334,  0.89443866]),
   array([ 0.65363971,  0.92425147,  0.35309849])]},
 'n_batches': 2,
 'n_sim': 2000}

In [7]:
# Continue inference from the previous state (with n_sim=2000)
custom_method.infer(n_sim=4000) # {'n_batches': 4, 'n_sim': 4000, 'filtered_outputs': ...}

{'filtered_outputs': {'d': [array([ 0.07274003,  0.05496166,  0.03369005,  0.02783118,  0.06705382]),
   array([ 0.04938172,  0.039332  ,  0.04218729]),
   array([ 0.05017163,  0.0777252 ,  0.08430152,  0.0835376 ]),
   array([ 0.01315948,  0.05887771,  0.02284609,  0.07160721,  0.05499586])],
  't1': [array([ 0.67284382,  0.76477027,  0.62571009,  0.46825312,  0.54522481]),
   array([ 0.60167879,  1.08463265,  0.8094352 ]),
   array([ 0.72408381,  0.36986854,  0.65311972,  0.56907475]),
   array([ 0.56842926,  0.67583018,  0.94228685,  0.62502374,  0.51659568])],
  't2': [array([ 0.54927791,  0.45852012,  0.27436906,  0.75507334,  0.89443866]),
   array([ 0.65363971,  0.92425147,  0.35309849]),
   array([ 0.49265925,  0.76532727,  0.42632809,  0.73731674]),
   array([ 0.16269547,  0.48695546,  0.44987195,  0.51514191,  0.20618901])]},
 'n_batches': 4,
 'n_sim': 4000}

In [8]:
# Or use it iteratively
custom_method.set_objective(n_sim=6000)

custom_method.iterate()
assert custom_method.finished == False
# Investigate the current state
custom_method.extract_result()  # {'n_batches': 5, 'n_sim': 5000, 'filtered_outputs': ...}

{'filtered_outputs': {'d': [array([ 0.07274003,  0.05496166,  0.03369005,  0.02783118,  0.06705382]),
   array([ 0.04938172,  0.039332  ,  0.04218729]),
   array([ 0.05017163,  0.0777252 ,  0.08430152,  0.0835376 ]),
   array([ 0.01315948,  0.05887771,  0.02284609,  0.07160721,  0.05499586]),
   array([ 0.03626514,  0.01565701,  0.09034186,  0.08375578,  0.09544032,
           0.06725741])],
  't1': [array([ 0.67284382,  0.76477027,  0.62571009,  0.46825312,  0.54522481]),
   array([ 0.60167879,  1.08463265,  0.8094352 ]),
   array([ 0.72408381,  0.36986854,  0.65311972,  0.56907475]),
   array([ 0.56842926,  0.67583018,  0.94228685,  0.62502374,  0.51659568]),
   array([ 0.64612478,  0.76469718,  0.38542154,  0.55239334,  0.38164821,
           0.53674665])],
  't2': [array([ 0.54927791,  0.45852012,  0.27436906,  0.75507334,  0.89443866]),
   array([ 0.65363971,  0.92425147,  0.35309849]),
   array([ 0.49265925,  0.76532727,  0.42632809,  0.73731674]),
   array([ 0.16269547,  0.48695

In [9]:
custom_method.iterate()
assert custom_method.finished
custom_method.extract_result()  # {'n_batches': 6, 'n_sim': 6000, 'filtered_outputs': ...}

{'filtered_outputs': {'d': [array([ 0.07274003,  0.05496166,  0.03369005,  0.02783118,  0.06705382]),
   array([ 0.04938172,  0.039332  ,  0.04218729]),
   array([ 0.05017163,  0.0777252 ,  0.08430152,  0.0835376 ]),
   array([ 0.01315948,  0.05887771,  0.02284609,  0.07160721,  0.05499586]),
   array([ 0.03626514,  0.01565701,  0.09034186,  0.08375578,  0.09544032,
           0.06725741]),
   array([ 0.0316558 ,  0.09233554,  0.04304713])],
  't1': [array([ 0.67284382,  0.76477027,  0.62571009,  0.46825312,  0.54522481]),
   array([ 0.60167879,  1.08463265,  0.8094352 ]),
   array([ 0.72408381,  0.36986854,  0.65311972,  0.56907475]),
   array([ 0.56842926,  0.67583018,  0.94228685,  0.62502374,  0.51659568]),
   array([ 0.64612478,  0.76469718,  0.38542154,  0.55239334,  0.38164821,
           0.53674665]),
   array([ 0.75710197,  0.72997056,  0.28815311])],
  't2': [array([ 0.54927791,  0.45852012,  0.27436906,  0.75507334,  0.89443866]),
   array([ 0.65363971,  0.92425147,  0.35309

# Extract the results

In [10]:
import numpy as np

from elfi.methods.parameter_inference import ParameterInference
from elfi.methods.results import Sample

class CustomMethod(ParameterInference):
    def __init__(self, model, discrepancy_name, threshold, **kwargs):
        # Create a name list of nodes whose outputs we wish to receive
        output_names = [discrepancy_name] + model.parameter_names
        super(CustomMethod, self).__init__(model, output_names, **kwargs)

        self.threshold = threshold
        self.discrepancy_name = discrepancy_name

        # Prepare lists to push the filtered outputs into
        self.state['filtered_outputs'] = {name: [] for name in output_names}

    def set_objective(self, n_sim):
        self.objective['n_sim'] = n_sim

    def update(self, batch, batch_index):
        super(CustomMethod, self).update(batch, batch_index)

        # Make a filter mask (logical numpy array) from the distance array
        filter_mask = batch[self.discrepancy_name] <= self.threshold

        # Append the filtered parameters to their lists
        for name in self.output_names:
            values = batch[name]
            self.state['filtered_outputs'][name].append(values[filter_mask])

    def extract_result(self):
        filtered_outputs = self.state['filtered_outputs']
        outputs = {name: np.concatenate(filtered_outputs[name]) for name in self.output_names}

        return Sample(
            method_name='CustomMethod',
            outputs=outputs,
            parameter_names=self.parameter_names,
            discrepancy_name=self.discrepancy_name,
            n_sim=self.state['n_sim'],
            threshold=self.threshold
            )

In [11]:
# Run the inference
custom_method = CustomMethod(m, 'd', threshold=.1, batch_size=1000)
custom_method.infer(n_sim=2000) 

Method: CustomMethod
Number of samples: 7
Number of simulations: 2000
Threshold: 0.1
Sample means: t1: 0.551, t2: 0.564